In [1]:
import pandas as pd
from datasets import load_dataset

In [2]:
metadata_raw = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "raw_meta_Video_Games", trust_remote_code=True)

In [3]:
metadata_raw_df = metadata_raw['full'].to_pandas()

In [4]:
metadata_raw_df

,main_category,title,average_rating,rating_number,features,description,price,images,videos,store,categories,details,parent_asin,bought_together,subtitle,author
0,Video Games,Dash 8-300 Professional Add-On,5.0,1,[Features Dash 8-300 and 8-Q300 ('Q' rollout l...,[The Dash 8-300 Professional Add-On lets you p...,None,"{'hi_res': [None], 'large': ['https://m.media-...","{'title': [], 'url': [], 'user_id': []}",Aerosoft,"[Video Games, PC, Games]","{""Pricing"": ""The strikethrough price is the Li...",B000FH0MHO,None,None,None
1,Video Games,Phantasmagoria: A Puzzle of Flesh,4.1,18,[Windows 95],[],None,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': [], 'url': [], 'user_id': []}",Sierra,"[Video Games, PC, Games]","{""Best Sellers Rank"": {""Video Games"": 137612, ...",B00069EVOG,None,None,None
2,Video Games,NBA 2K17 - Early Tip Off Edition - PlayStation 4,4.3,223,[The #1 rated NBA video game simulation series...,[Following the record-breaking launch of NBA 2...,58.0,{'hi_res': ['https://m.media-amazon.com/images...,{'title': ['NBA 2K17 - Kobe: Haters vs Players...,2K,"[Video Games, PlayStation 4, Games]","{""Release date"": ""September 16, 2016"", ""Best S...",B00Z9TLVK0,None,None,None
3,Video Games,Nintendo Selects: The Legend of Zelda Ocarina ...,4.9,22,[Authentic Nintendo Selects: The Legend of Zel...,[],37.42,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': [], 'url': [], 'user_id': []}",Amazon Renewed,"[Video Games, Legacy Systems, Nintendo Systems...","{""Best Sellers Rank"": {""Video Games"": 51019, ""...",B07SZJZV88,None,None,None
4,Video Games,Thrustmaster Elite Fitness Pack for Nintendo Wii,3.0,3,"[Includes (9) Total Accessories, Pedometer, Wi...",[The Thrustmaster Motion Plus Elite Fitness Pa...,None,"{'hi_res': [None, None, None, None, None, None...","{'title': [], 'url': [], 'user_id': []}",THRUSTMASTER,"[Video Games, Legacy Systems, Nintendo Systems...","{""Release date"": ""November 1, 2009"", ""Pricing""...",B002WH4ZJG,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137264,None,Story of Seasons: Pioneers Of Olive Town (Nint...,4.5,397,[A wild world of discovery - tame the wilderne...,"[Product Description, Inspired by Tales of you...",31.04,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': [], 'url': [], 'user_id': []}",Marvelous Europe,"[Video Games, Nintendo Switch, Games]","{""Release date"": ""March 26, 2021"", ""Best Selle...",B09XQJS4CZ,None,None,None
137265,Video Games,MotoGP 18 (PC DVD) UK IMPORT REGION FREE,4.0,1,[Brand new game engine - MotoGP18 has been reb...,[Become the champion of the 2018 MotoGP Season...,None,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': [], 'url': [], 'user_id': []}",Milestone,"[Video Games, Game Genre of the Month]","{""Pricing"": ""The strikethrough price is the Li...",B07DGPTGNV,None,None,None
137266,Cell Phones & Accessories,Century Accessory Soft Silicone Protective Ski...,2.9,19,"[Easy access to all buttons, controls and port...",[This soft case cover will add a splash of col...,None,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': [], 'url': [], 'user_id': []}",Century Accessory,"[Video Games, Legacy Systems, Xbox Systems, Xb...","{""Package Dimensions"": ""2.76 x 2.76 x 0.2 inch...",B00HUWCQBW,None,None,None
137267,None,Hasbro Interactive Mr. Potato Head Activity Pa...,3.9,5,[],"[Amazon.com, Everyone's favorite master-of-dis...",None,"{'hi_res': [None, 'https://m.media-amazon.com/...","{'title': [], 'url': [], 'user_id': []}",Hasbro,"[Video Games, PC, Games]","{""Release date"": ""July 24, 1999"", ""Best Seller...",B00002S9MH,None,None,None


In [5]:
with pd.option_context('display.max_colwidth', None):
    display(metadata_raw_df.sample(10))

main_category  \
135766          None   
91317    Video Games   
119171   Video Games   
58945    Video Games   
26287      Computers   
135654   Video Games   
104424   Video Games   
96244    Video Games   
7571     Video Games   
77946    Video Games   

                                                                                                                                                                                           title  \
135766                                                                                             NBA - Noche Latina Jersey - Miami El Heat - Skin for 1 Microsoft Xbox 360 Wireless Controller   
91317                                                                                                                                                             Universal Combat CE [Download]   
119171                                                                                                                                                    Total War: Attila (PC DVD) (UK IMPORT)   
58945                                                                                                                                                            Watch Dogs Limited Edition - PC   
26287                                     SADES SA902 7.1 Surround Sound PC Gaming Headset USB Over Ear Headphones with Microphone LED Light Volume Control for Mac/Computer/Laptop - Black/Blue   
135654                                                                                                                                            Army of Two: The 40th Day - Xbox 360 (Renewed)   
104424                                                                                                                                                                               Paper Mario   
96244                                                                                                                                                                   MediEvil - PlayStation 4   
7571    Minneer Racing Steering Wheel Cockpit Pro, Height Adjustable Gaming Wheel Stand for Logitech G25 G27 G29 G920 G923 and Thrustmaster T300RS & TX F458 & T500R Wheel & Pedals Not Included   
77946                                                                                                                                                                Temple of Jewels [Download]   

        average_rating  rating_number  \
135766             3.3              2   
91317              3.1             18   
119171             3.8              4   
58945              3.8             22   
26287              4.1             22   
135654             3.3              6   
104424             4.2            327   
96244              4.7           3337   
7571               4.3             79   
77946              1.6              4   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

# Build Feature Transformations

In [6]:
train_df = pd.read_parquet("../data/train.parquet")
val_df = pd.read_parquet("../data/val.parquet")

In [7]:
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import FunctionTransformer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.pipeline import Pipeline

In [8]:
# Custom function to reshape data from 1D to 2D
def reshape_2d_to_1d(X):
    return np.array(X).reshape(-1)

def flatten_string_array_col(X):
    return np.array(['\n'.join(x) for x in X])

def tfidf_pipeline_steps():
    steps = [
        ('impute', SimpleImputer(strategy='constant', fill_value='')),
        ('reshape', FunctionTransformer(reshape_2d_to_1d, validate=False)),
        ('tfidf', TfidfVectorizer(min_df=5, max_features=1000, ngram_range=(1, 2)))
    ]
    return steps

def description_pipeline_steps():
    steps = [
        ('flatten_string_array_col', FunctionTransformer(flatten_string_array_col, validate=False)),
        ('tfidf', TfidfVectorizer(min_df=5, max_features=1000, ngram_range=(1, 2)))
    ]
    return steps

def categories_pipeline_steps():
    def tokenizer(s):
        return s.split('\n')
    steps = [
        ('flatten_string_array_col', FunctionTransformer(flatten_string_array_col, validate=False)),
        ('tfidf', CountVectorizer(tokenizer=tokenizer, token_pattern=None))
    ]
    return steps

data = pd.Series(["from 14.99", "14.99", "price: 9.99", "20 dollars", "none"])
price_pattern = r'\b((?:\d+\.\d*)|(?:\d+))\b'
display(data.str.extract(price_pattern))

def price_parse_dtype(series, pattern):
    return series.str.extract(pattern).astype(float)

def price_pipeline_steps(price_pattern):
    steps = [
        ('extract_price', FunctionTransformer(price_parse_dtype, kw_args=dict(pattern=price_pattern), validate=False)),
        ('impute', SimpleImputer(strategy='constant', fill_value=0)),
        ('min_max_scale', MinMaxScaler())
    ]
    return steps

,0
0,14.99
1,14.99
2,9.99
3,20
4,NaN


In [9]:
tfm = [
    ('main_category', OneHotEncoder(handle_unknown='ignore'), ['main_category']),  # One-hot encoding for categorical field
    ('title', Pipeline(tfidf_pipeline_steps()), ['title']),  # TF-IDF vectorizer for text field
    ('description', Pipeline(description_pipeline_steps()), 'description'),  # TF-IDF vectorizer for another text field
    ('categories', Pipeline(categories_pipeline_steps()), 'categories'),  # Count Vectorizer for multi-label categorical
    ('price', Pipeline(price_pipeline_steps(price_pattern)), 'price')  # Normalizing price
]
cols = [x[0] for x in tfm]
cols

['main_category', 'title', 'description', 'categories', 'price']

In [10]:
train_full_df = pd.merge(train_df, metadata_raw_df[['parent_asin'] + cols], how='left', on='parent_asin')
# Drop duplicated item so that the Pipeline only fit the unique item features
fit_df = train_full_df.drop_duplicates(subset=['parent_asin'])
val_full_df = pd.merge(val_df, metadata_raw_df[['parent_asin'] + cols], how='left', on='parent_asin')
train_full_df

,user_id,parent_asin,rating,timestamp,main_category,title,description,categories,price
0,AFFZVSTUS3U2ZD22A2NPZSKOCPGQ,B01GW3LRD2,5.0,1491589434000,Video Games,Forza Horizon 3 – Xbox One,[You’re in charge of the Horizon Festival. Cus...,"[Video Games, Xbox One, Games]",22.0
1,AFFZVSTUS3U2ZD22A2NPZSKOCPGQ,B0848LKV51,4.0,1574659954094,Video Games,"RIG 400HX Stereo Gaming Headset for Xbox One, ...",[Gear up to hear more detail in your games. RI...,"[Video Games, Xbox One, Accessories, Headsets]",None
2,AF4WLLHTQLRPEZ33OJDYG23MFLKQ,B09BQ4ZDQZ,5.0,1553570826020,Computers,HUO JI E-Yooso Z-88 Typewriter Mechanical Keyb...,"[★ A Fusion of Retro Style and Modern Design, ...","[Video Games, PC, Accessories, Gaming Keyboards]",22.99
3,AF4WLLHTQLRPEZ33OJDYG23MFLKQ,B01MDQP1ZU,5.0,1561162491649,Video Games,Amazon Basics AC Charger Adapter for Nintendo ...,"[Product Description, Amazon Basics AC Charger...","[Video Games, Legacy Systems, Nintendo Systems...",None
4,AF4WLLHTQLRPEZ33OJDYG23MFLKQ,B0171AOQG0,5.0,1561266333491,Computers,Redragon M990 Legend 24000 DPI High-Precision ...,"[Redragon M990 RGB ""Legend Chrome"" Gaming Mous...","[Video Games, PC, Accessories, Gaming Mice]",None
...,...,...,...,...,...,...,...,...,...
36938,AGRXRIPAZTGAQHFKXZLFJDUOJSJA,B08FRMGWXQ,5.0,1609218633016,Video Games,eXtremeRate 2 Pack Black Border Transparent HD...,[],"[Video Games, Nintendo Switch, Accessories, Fa...",14.59
36939,AEFPHMM7CLX4UJNXJFQF4ZF5GNAA,B01BO2012O,3.0,1554983583540,Computers,HDE Travel Bag Bundle for Nintendo 3DS Travel ...,[],"[Video Games, Legacy Systems, Nintendo Systems...",None
36940,AEFPHMM7CLX4UJNXJFQF4ZF5GNAA,B001NJMMHG,5.0,1599584891516,Video Games,Dragon Quest V: Hand of the Heavenly Bride - N...,"[Product Description, DRAGON QUEST V: Hand of ...","[Video Games, Legacy Systems, Nintendo Systems...",309.99
36941,AEFPHMM7CLX4UJNXJFQF4ZF5GNAA,B07P27XFP7,5.0,1599585146628,Video Games,Shin Megami Tensei: Strange Journey Redux - Ni...,"[""A spatial distortion full of demons appeared...","[Video Games, Legacy Systems, Nintendo Systems...",69.95


In [11]:
# Define preprocessing steps for each column
preprocessing_pipeline = ColumnTransformer(
    transformers=tfm,
    remainder='drop'  # Drop any columns not specified in transformers
)

# Create a pipeline object
item_metadata_pipeline = Pipeline(steps=[
    ('preprocessing', preprocessing_pipeline)
])

# Fit the pipeline
item_metadata_pipeline.fit(fit_df)

# Transform the data (useful for training)
transformed_item_metadata = item_metadata_pipeline.transform(train_full_df)

# For demonstration, print the shape of the transformed features and a few rows
print(f"Transformed Item Metadata Shape: {transformed_item_metadata.shape}")

Transformed Item Metadata Shape: (36943, 2180)


# Persist

In [12]:
import joblib
import pickle

In [13]:
train_full_df.to_parquet("../data/train_item_features.parquet")
val_full_df.to_parquet("../data/val_item_features.parquet")

In [14]:
joblib.dump(item_metadata_pipeline, '../data/item_metadata_pipeline.joblib')

['../data/item_metadata_pipeline.joblib']